<a href="https://colab.research.google.com/github/byeungchun/cbspeech_topicmodelling/blob/main/topicmodelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code example: Large language models: a primer for economists

In this notebook, we analyze the factors influencing stock market sentiment using Central Bank speeches from the Bank for International Settlements website, spanning 2014 to 2022.

The data processing pipeline includes:
- <b>Segmentation</b>: Dividing speeches into smaller, manageable chunks.

- <b>Tokenization</b>: Converting text into tokens for each chunk.

- <b>Embedding Transformation</b>: Mapping each chunk into numerical vectors (embeddings) to capture semantic meaning.

- <b>Clustering and Topic Extraction</b>: Grouping embeddings to identify thematic clusters, with topics based on frequently co-occurring terms.

## Setup

In [1]:
# pip install bertopic
!pip install bertopic accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.0 MB/s eta 0:00:00


In [35]:
import re
import json
import requests
import pandas as pd
import numpy as np
import torch

from tqdm import tqdm
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer

from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired

### Topic Modeling Parameters

- **wikipedia_jsonfile**: URL to the JSON file containing 10 wikipedia pages. This file is hosted on GitHub and will be used as input for topic modeling.
  
- **word_chunk_size**: Defines the size of word chunks to be used in the topic model. Smaller values may capture more fine-grained themes, while larger values may generalize more.

- **bertopic_nr_topics**: Specifies the number of topics to generate in BERTopic modeling. This parameter controls the level of granularity in the topic clusters.

- **embeddingmodel**: Specifies the Hugging Face embedding model to be used for text representations. `'microsoft/Phi-3-mini-4k-instruct'` is a pre-trained model suitable for NLP tasks, but feel free to replace this with any other 'text generation' model available on Hugging Face.


In [4]:
speech_jsonfile = r'https://github.com/byeungchun/cbspeech_topicmodelling/blob/main/wikipedia_10pages.json?raw=true'
word_chunk_size = 100
bertopic_nr_topics = 7
embeddingmodel = 'microsoft/Phi-3-mini-4k-instruct'


## Data Loading
10 Wikipedia pages

In [5]:
# Fetch the data from the URL
response = requests.get(speech_jsonfile)
response.raise_for_status()  # Raise an exception for bad status codes

# Load the JSON data
data = json.loads(response.text)
df = pd.DataFrame(data)

In [6]:
df.head()

,title,text,url
0,Bank for International Settlements,The Bank for International Settlements (BIS) i...,https://en.wikipedia.org/wiki/Bank_for_Interna...
1,United States,"The United States of America (USA), commonly k...",https://en.wikipedia.org/wiki/United_States
2,Machine Learning,Machine learning (ML) is a field of study in a...,https://en.wikipedia.org/wiki/Machine_learning
3,Data Science,Data science is an interdisciplinary academic ...,https://en.wikipedia.org/wiki/Data_science
4,Artificial Intelligence,"Artificial intelligence (AI), in its broadest ...",https://en.wikipedia.org/wiki/Artificial_intel...


## Data Processing
- **Chunking:** Split each Wikipedia page into chunks of 100 words each.
- **Embedding Generation:** Create an embedding array for each chunk.

### Chunking

In [7]:
# Preprocess document content and generate text chunks for analysis
# - docs: List to store unique text chunks extracted from the documents
# - df_docs_mapper: List to map the original text ID to generated document chunks
# - docs_cnt: Counter to assign unique IDs to document chunks

# Initialize variables for document processing
docs = []
df_docs_mapper = []
docs_cnt = 0

# Iterate through each record in the DataFrame and generate text chunks
for i, rec in tqdm(df.iterrows(), total=len(df)):
    text = rec['text']
    text2 = re.sub(r'[\.\n]', ' ', text)  # Replace periods and newlines with space
    text2 = re.sub(r'\s+', ' ', text2)  # Replace multiple spaces with a single space
    text2 = re.sub(r'[^a-zA-Z0-9 ]', '', text2).strip()  # Remove non-alphanumeric characters
    words = text2.split()

    # Create word chunks from the cleaned text
    word_idx = 0
    while word_idx < len(words):
        word_piece = words[word_idx: word_idx + word_chunk_size]
        is_piece_chunksize = len(word_piece) == word_chunk_size
        chunk_text = ' '.join(word_piece)

        # Add unique chunks to docs and update the document mapper
        if is_piece_chunksize and chunk_text not in docs:
            docs.append(chunk_text)
            df_docs_mapper.append((i, docs_cnt))
            docs_cnt += 1
        word_idx = word_idx + word_chunk_size

# Convert document mapper list to DataFrame and merge with document chunks
df_docs_mapper = pd.DataFrame(df_docs_mapper, columns=['text_id', 'doc_id'])
df_docs = df_docs_mapper.merge(pd.DataFrame(docs), left_index=True, right_index=True)

100%|██████████| 10/10 [00:00<00:00, 106.20it/s]


In [8]:
print(f'Total number of pages: {len(df_docs_mapper.text_id.drop_duplicates())}, total number of chunks (each chunk contains {word_chunk_size} words): {len(docs)}')


Total number of pages: 10, total number of chunks (each chunk contains 100 words): 768


In [9]:
chunk_stat = df_docs_mapper.groupby('text_id').count().describe()
print(f'For each page, the average number of chunks is: {chunk_stat.iloc[1,0]:.2f}, with a minimum of {chunk_stat.iloc[3,0]} chunks and a maximum of {chunk_stat.iloc[7,0]} chunks.')


For each page, the average number of chunks is: 76.80, with a minimum of 10.0 chunks and a maximum of 132.0 chunks.


In [10]:
# Merge document chunks with mapper and publication dates
# - df_docs: DataFrame that contains document chunks, their corresponding IDs, and publication dates

# Merge document chunks with document mapper DataFrame
df_docs = df_docs_mapper.merge(pd.DataFrame(docs), how='inner', left_on='doc_id', right_index=True)
df_docs.columns = ['text_id', 'doc_id', 'chunk']

### Embedding

#### Embedding Generation Process

- **Purpose**: Generate sentence embeddings using a pre-trained language model.
- **get_sentence_embedding**: A function that generates the embedding for a given sentence using a tokenizer and model.
- **embeddings**: A list to store the embeddings for all documents in the dataset.
- **CPU vs. GPU**:
  - If the code is executed on a **CPU**, it loads precomputed embeddings from a JSON file hosted on GitHub because CPU performance is slower for embedding generation, making real-time processing inefficient.
  - If on **GPU**, it calculates embeddings in real-time using the specified model and tokenizer.

- **get_sentence_embedding Function**:
  - Takes a sentence as input, tokenizes it, and passes it through the model.
  - Extracts the last hidden state and computes the mean to produce the sentence embedding.

- **Final Step**: Converts the embeddings list into a numpy array for further processing.


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == 'cpu':
  import io
  npzfileurl = 'https://github.com/byeungchun/cbspeech_topicmodelling/blob/main/embeddings_msphi3.npz?raw=true'
  response = requests.get(npzfileurl)
  response.raise_for_status()  # Check for errors
  with np.load(io.BytesIO(response.content)) as data:
    embeddings = data['array']
else:
  llm_engine = AutoModel.from_pretrained(
      embeddingmodel,
      device_map= 'auto',
      torch_dtype=torch.float16,
  ).to(device)
  tokenizer = AutoTokenizer.from_pretrained(embeddingmodel)

  def get_sentence_embedding(sentence):
      inputs = tokenizer(sentence, return_tensors='pt').to(device)
      with torch.no_grad():
          outputs = llm_engine(**inputs)
      last_hidden_state = outputs.last_hidden_state
      sentence_embedding = torch.mean(last_hidden_state, dim=1).squeeze().tolist()
      return sentence_embedding

  embeddings = [get_sentence_embedding(doc) for doc in tqdm(docs)]
  embeddings = np.array(embeddings)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

100%|██████████| 768/768 [00:59<00:00, 12.91it/s]


## Topic Modeling

BERTopic model to identify key themes in the dataset. Key parameters include:

- **Top Words (`num_top_words`):** Number of key terms per topic.
- **KeyBERT Words (`num_keybert_words`):** Words representing each topic using KeyBERT.
- **Topic Limit (`topic_limit`):** Maximum number of topics.



In [49]:
num_top_words = 30
num_keybert_words = 10
topic_limit = 20
min_cluster_size = 50
n_gram_range = (1, 3)
nr_topics = 20

representation_model = {"KeyBERT": KeyBERTInspired(top_n_words=num_keybert_words, nr_repr_docs=20)}
vectorizer_model = CountVectorizer(stop_words="english")

# Step 1 - Extract embeddings
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=10,  # Neighboring sample points (Increasing means more global view)
                  n_components=10, # Reduced dimensions of the embeddings
                  min_dist=0.0,
                  metric='euclidean')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size,
                        metric='euclidean',
                        cluster_selection_method='eom',
                        prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english",  # Removes stopwords
                                    ngram_range=n_gram_range)  # No. of words in a topic

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with
# a `bertopic.representation` model
representation_model = {
  "KeyBERT": KeyBERTInspired(
    top_n_words=15,  # The top n words to extract per topic (default 10)
    nr_repr_docs=12,  # The number of representative documents to extract per cluster (default 5)
    nr_samples=300  # The number of candidate documents to extract per cluster (default 500)
  )
}

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  nr_topics=nr_topics,                      # Maximum number of topics
  n_gram_range=n_gram_range                 # No. of words per topic
)
# Fit and transform the model on the documents and embeddings
topics, initial_probabilities = topic_model.fit_transform(docs, embeddings)

# Create a DataFrame with the topics and their corresponding probabilities
df_results = pd.DataFrame({'Topic': topics, 'Probability': initial_probabilities})

# Get detailed topic information from the BERTopic model and merge with the results DataFrame
df_topic_info = topic_model.get_topic_info()
df_results = df_results.merge(df_topic_info, how='left', on='Topic')

# Merge document mapper with results DataFrame and filter out documents without topics
embedding_list = [list(val) for val in embeddings]
df_results = df_docs_mapper.merge(df_results, how='left', left_on='doc_id', right_index=True)
df_bertopic_results = df_results[df_results.Topic != -1.0].dropna(how='any').reset_index(drop=True)

In [50]:
df_results['KeyBERT'] = df_results['KeyBERT'].apply(tuple)
topic_keybert_counts = df_results.groupby(['Topic', 'KeyBERT']).size().reset_index(name='counts')
topic_keybert_counts

,Topic,KeyBERT,counts
0,-1,"(banking crises, financial crises, financial c...",20
1,0,"(machine learning, learning algorithms, artifi...",224
2,1,"(olympic marathon, marathon held, marathon dis...",70
3,2,"(united states, states, america, federal gover...",142
4,3,"(global financial, international financial, ce...",312


### (Option) LDA Topic Modeling

This optional step applies LDA to identify topics across document chunks. While human labeling was used for topic identification in the main analysis, LDA provides a complementary, automated approach to confirm and explore additional themes. Key parameters and components include:

- **LDA Topic Results (`lda_topic_results`):** Stores LDA topic modeling outcomes for each document set.
- **Topics to Extract (`num_topics_to_extract`):** Specifies the number of topics to identify in each LDA model.

In [52]:
lda_topic_results = []
num_topics_to_extract = 2

# Prepare the documents and dictionary for LDA modeling
document_texts = df_bertopic_results.KeyBERT.to_list()
doc_dictionary = corpora.Dictionary(document_texts)
bow_corpus = [doc_dictionary.doc2bow(doc) for doc in document_texts]

# Train the LDA model with the specified number of topics
lda_model = LdaModel(corpus=bow_corpus, id2word=doc_dictionary, num_topics=num_topics_to_extract, passes=10)

# Extract topics for each document and store the results
document_topic_info = []
for idx, doc_bow in enumerate(bow_corpus):
    doc_topics = lda_model.get_document_topics(doc_bow, minimum_probability=0.0)
    doc_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)
    dominant_topic = doc_topics[0][0]
    document_topic_info.append({
        'lda_topic': dominant_topic,
        'lda_probability': doc_topics[0][1],
        'lda_words': ' '.join([x[0] for x in sorted(lda_model.show_topic(dominant_topic), reverse=False)])
    })
lda_topic_results.append(document_topic_info)

# Print summary of LDA modeling results
print(f'The LDA model was trained with {num_topics_to_extract} topics.')


The LDA model was trained with 2 topics.


In [53]:
df_bertopic_results.head()

,text_id,doc_id,Topic,Probability,Count,Name,Representation,KeyBERT,Representative_Docs
0,0,0,3,1.0,312,3_financial_bank_economics_economic,"[financial, bank, economics, economic, banks, ...","[global financial, international financial, ce...",[national governments and intergovernmental or...
1,0,1,3,1.0,312,3_financial_bank_economics_economic,"[financial, bank, economics, economic, banks, ...","[global financial, international financial, ce...",[national governments and intergovernmental or...
2,0,2,3,1.0,312,3_financial_bank_economics_economic,"[financial, bank, economics, economic, banks, ...","[global financial, international financial, ce...",[national governments and intergovernmental or...
3,0,3,3,1.0,312,3_financial_bank_economics_economic,"[financial, bank, economics, economic, banks, ...","[global financial, international financial, ce...",[national governments and intergovernmental or...
4,0,4,3,1.0,312,3_financial_bank_economics_economic,"[financial, bank, economics, economic, banks, ...","[global financial, international financial, ce...",[national governments and intergovernmental or...


In [54]:
df_bertopic_results['KeyBERT'] = df_bertopic_results['KeyBERT'].apply(tuple)
df_bertopic_results[['Name','Count','KeyBERT']].drop_duplicates().sort_values(by='Count',ascending=False)

,Name,Count,KeyBERT
0,3_financial_bank_economics_economic,312,"(global financial, international financial, ce..."
189,0_learning_data_ai_machine,224,"(machine learning, learning algorithms, artifi..."
57,2_states_united_united states_american,142,"(united states, states, america, federal gover..."
186,1_marathon_marathons_runners_run,70,"(olympic marathon, marathon held, marathon dis..."


In [55]:
lda_topic_results = pd.DataFrame(document_topic_info)

# Print summary of LDA modeling results
print(f'The LDA model was trained with {num_topics_to_extract} topics. The number of documents processed is {len(lda_topic_results)}.')

# Merge BERTopic results and LDA topic results into a single DataFrame
merged_results = df_bertopic_results.merge(lda_topic_results, how='left', left_index=True, right_index=True)

The LDA model was trained with 2 topics. The number of documents processed is 748.


In [56]:
merged_results[['lda_topic','lda_words']].drop_duplicates()

,lda_topic,lda_words
0,1,banks central bank currencies currency economi...
57,0,america american congress constitution countri...
